#### Домашнее задание по теме «Рекомендации на основе содержания»

* Использовать dataset MovieLens
* Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
* Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
links = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\links.csv')
movies = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\movies.csv')
ratings = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\ratings.csv')
tags = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
data1 = movies.join(tags.set_index('movieId'), on='movieId')    # movies.merge(tags)
data1.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [8]:
data2 = data1.join(ratings.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')
# data1.merge(ratings.set_index('movieId'), on='movieId')
data2.head()

,movieId,title,genres,userId_left,tag,timestamp_left,userId_right,rating,timestamp_right
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,1.0,4.0,9.649827e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,5.0,4.0,8.474350e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,7.0,4.5,1.106636e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,15.0,2.5,1.510578e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,17.0,4.5,1.305696e+09


In [9]:
data = data2[['movieId', 'genres', 'userId_left', 'tag', 'userId_right', 'rating']]
data.head()

,movieId,genres,userId_left,tag,userId_right,rating
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5


In [10]:
data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.lower().replace(' ', '').replace('-', '').split('|')))
data.head()

<ipython-input-10-5eed5cbd4619>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.lower().replace(' ', '').replace('-', '').split('|')))


,movieId,genres,userId_left,tag,userId_right,rating,Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,adventure animation children comedy fantasy


In [11]:
data['tag'] = data['tag'].apply(lambda x: str(x).lower())
data.head()

<ipython-input-11-3919233e2269>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tag'] = data['tag'].apply(lambda x: str(x).lower())


,movieId,genres,userId_left,tag,userId_right,rating,Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,adventure animation children comedy fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,adventure animation children comedy fantasy


In [12]:
data = data.drop('genres', axis=1)
data.head()

,movieId,userId_left,tag,userId_right,rating,Genre
0,1,336.0,pixar,1.0,4.0,adventure animation children comedy fantasy
0,1,336.0,pixar,5.0,4.0,adventure animation children comedy fantasy
0,1,336.0,pixar,7.0,4.5,adventure animation children comedy fantasy
0,1,336.0,pixar,15.0,2.5,adventure animation children comedy fantasy
0,1,336.0,pixar,17.0,4.5,adventure animation children comedy fantasy


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285783 entries, 0 to 9741
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       285783 non-null  int64  
 1   userId_left   233234 non-null  float64
 2   tag           285783 non-null  object 
 3   userId_right  285762 non-null  float64
 4   rating        285762 non-null  float64
 5   Genre         285783 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 15.3+ MB


In [14]:
data.dropna(inplace=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 9732
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       233213 non-null  int64  
 1   userId_left   233213 non-null  float64
 2   tag           233213 non-null  object 
 3   userId_right  233213 non-null  float64
 4   rating        233213 non-null  float64
 5   Genre         233213 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 12.5+ MB


##### Выведем метрику на каждый фильм и добавим его в фичи

In [16]:
# посчитаем среднюю оценку на каждый фильм
mean_score_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    mean_score_mov[title] = group.rating.mean()

In [17]:
# посчитаем по каждому фильму количество рейтингов
num_action_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    num_action_mov[title] = group.shape[0]

In [18]:
# метрики
min_num_action_mov = np.min([num_action_mov[f] for f in num_action_mov.keys()])
max_num_action_mov = np.max([num_action_mov[f] for f in num_action_mov.keys()])
mean_num_action_mov = np.mean([num_action_mov[f] for f in num_action_mov.keys()])

In [19]:
# посчитаем метрику фильма
score_mov = {}

for f in num_action_mov.keys():
        score_mov[f] = (mean_score_mov[f] * (num_action_mov[f] - mean_num_action_mov)/
                        (max_num_action_mov - min_num_action_mov))

In [20]:
# добавим признак - метрику фильма

data['score_mov'] = data['movieId'].apply(lambda x: score_mov[x])
data.head()

,movieId,userId_left,tag,userId_right,rating,Genre,score_mov
0,1,336.0,pixar,1.0,4.0,adventure animation children comedy fantasy,0.034924
0,1,336.0,pixar,5.0,4.0,adventure animation children comedy fantasy,0.034924
0,1,336.0,pixar,7.0,4.5,adventure animation children comedy fantasy,0.034924
0,1,336.0,pixar,15.0,2.5,adventure animation children comedy fantasy,0.034924
0,1,336.0,pixar,17.0,4.5,adventure animation children comedy fantasy,0.034924


##### Посчитаем среднюю оценку каждого userId и добавим в фичи

In [21]:
mean_mark = {}

for title, group in tqdm(data.groupby('userId_right')):
    mean_mark[title] = group.rating.mean()

In [22]:
data['mean_mark'] = data['userId_right'].apply(lambda x: mean_mark[x])
data.head()

,movieId,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark
0,1,336.0,pixar,1.0,4.0,adventure animation children comedy fantasy,0.034924,4.040472
0,1,336.0,pixar,5.0,4.0,adventure animation children comedy fantasy,0.034924,4.517986
0,1,336.0,pixar,7.0,4.5,adventure animation children comedy fantasy,0.034924,3.672316
0,1,336.0,pixar,15.0,2.5,adventure animation children comedy fantasy,0.034924,3.601810
0,1,336.0,pixar,17.0,4.5,adventure animation children comedy fantasy,0.034924,4.538404


In [23]:
Tag = {}
for title, group in tqdm(data.groupby('movieId')):
    Tag[title] = ' '.join(group.tag.unique().tolist())
Tag

{1: 'pixar fun',
 2: 'fantasy magic board game robin williams game',
 3: 'moldy old',
 5: 'pregnancy remake',
 7: 'remake',
 11: 'politics president',
 14: 'politics president',
 16: 'mafia',
 17: 'jane austen',
 21: 'hollywood',
 22: 'serial killer',
 25: 'alcoholism',
 26: 'shakespeare',
 28: 'in netflix queue jane austen',
 29: 'kidnapping',
 31: 'high school teacher',
 32: 'time travel brad pitt bruce willis mindfuck post apocalyptic post-apocalyptic remake twist ending',
 34: 'animal movie pigs villain nonexistent or not needed for good story',
 36: 'death penalty nun',
 38: 'twins',
 39: 'chick flick funny paul rudd quotable seen more than once emma jane austen',
 40: 'in netflix queue south africa',
 41: 'shakespeare',
 43: 'england',
 45: 'journalism',
 46: 'wedding',
 47: 'mystery twist ending serial killer',
 50: 'mindfuck suspense thriller tricky twist ending heist',
 52: 'adoption prostitution',
 58: 'writing',
 62: 'music',
 92: 'jekyll and hyde',
 96: 'theater',
 101: 'cr

In [24]:
tag = list(Tag.values())
tag

['pixar fun',
 'fantasy magic board game robin williams game',
 'moldy old',
 'pregnancy remake',
 'remake',
 'politics president',
 'politics president',
 'mafia',
 'jane austen',
 'hollywood',
 'serial killer',
 'alcoholism',
 'shakespeare',
 'in netflix queue jane austen',
 'kidnapping',
 'high school teacher',
 'time travel brad pitt bruce willis mindfuck post apocalyptic post-apocalyptic remake twist ending',
 'animal movie pigs villain nonexistent or not needed for good story',
 'death penalty nun',
 'twins',
 'chick flick funny paul rudd quotable seen more than once emma jane austen',
 'in netflix queue south africa',
 'shakespeare',
 'england',
 'journalism',
 'wedding',
 'mystery twist ending serial killer',
 'mindfuck suspense thriller tricky twist ending heist',
 'adoption prostitution',
 'writing',
 'music',
 'jekyll and hyde',
 'theater',
 'crime off-beat comedy quirky',
 'test tag golf adam sandler',
 'muppets',
 'beautiful scenery epic historical inspirational medieval m

In [25]:
tfidf1 = TfidfVectorizer()

In [26]:
tfidf1.fit(tag)

TfidfVectorizer()

In [27]:
Genre = {}
for title, group in tqdm(data.groupby('movieId')):
    Genre[title] = ' '.join(group.Genre.unique().tolist())
Genre

{1: 'adventure animation children comedy fantasy',
 2: 'adventure children fantasy',
 3: 'comedy romance',
 5: 'comedy',
 7: 'comedy romance',
 11: 'comedy drama romance',
 14: 'drama',
 16: 'crime drama',
 17: 'drama romance',
 21: 'comedy crime thriller',
 22: 'crime drama horror mystery thriller',
 25: 'drama romance',
 26: 'drama',
 28: 'drama romance',
 29: 'adventure drama fantasy mystery scifi',
 31: 'drama',
 32: 'mystery scifi thriller',
 34: 'children drama',
 36: 'crime drama',
 38: 'children comedy',
 39: 'comedy romance',
 40: 'drama',
 41: 'drama war',
 43: 'drama',
 45: 'comedy drama thriller',
 46: 'drama romance',
 47: 'mystery thriller',
 50: 'crime mystery thriller',
 52: 'comedy drama romance',
 58: 'comedy drama romance',
 62: 'drama',
 92: 'drama horror thriller',
 96: 'comedy drama',
 101: 'adventure comedy crime romance',
 104: 'comedy',
 107: 'adventure children comedy musical',
 110: 'action drama war',
 111: 'crime drama thriller',
 116: 'documentary',
 122: 

In [28]:
genre = list(Genre.values())
genre

['adventure animation children comedy fantasy',
 'adventure children fantasy',
 'comedy romance',
 'comedy',
 'comedy romance',
 'comedy drama romance',
 'drama',
 'crime drama',
 'drama romance',
 'comedy crime thriller',
 'crime drama horror mystery thriller',
 'drama romance',
 'drama',
 'drama romance',
 'adventure drama fantasy mystery scifi',
 'drama',
 'mystery scifi thriller',
 'children drama',
 'crime drama',
 'children comedy',
 'comedy romance',
 'drama',
 'drama war',
 'drama',
 'comedy drama thriller',
 'drama romance',
 'mystery thriller',
 'crime mystery thriller',
 'comedy drama romance',
 'comedy drama romance',
 'drama',
 'drama horror thriller',
 'comedy drama',
 'adventure comedy crime romance',
 'comedy',
 'adventure children comedy musical',
 'action drama war',
 'crime drama thriller',
 'documentary',
 'comedy romance',
 'drama romance',
 'drama',
 'adventure drama imax',
 'action adventure comedy crime',
 'action adventure mystery scifi',
 'drama thriller war',

In [29]:
tfidf2 = TfidfVectorizer()

In [30]:
tfidf2.fit(genre)

TfidfVectorizer()

In [31]:
data['TAG'] = data['movieId'].apply(lambda x: Tag[x])
data['GENRE'] = data['movieId'].apply(lambda x: Genre[x])
data = data.drop(columns=['userId_left', 'tag', 'Genre'])
data.head()

,movieId,userId_right,rating,score_mov,mean_mark,TAG,GENRE
0,1,1.0,4.0,0.034924,4.040472,pixar fun,adventure animation children comedy fantasy
0,1,5.0,4.0,0.034924,4.517986,pixar fun,adventure animation children comedy fantasy
0,1,7.0,4.5,0.034924,3.672316,pixar fun,adventure animation children comedy fantasy
0,1,15.0,2.5,0.034924,3.601810,pixar fun,adventure animation children comedy fantasy
0,1,17.0,4.5,0.034924,4.538404,pixar fun,adventure animation children comedy fantasy


In [32]:
data_X = data[data['userId_right']==1].drop_duplicates(subset=['movieId'], keep='first')
data_X.head()

,movieId,userId_right,rating,score_mov,mean_mark,TAG,GENRE
0,1,1.0,4.0,0.034924,4.040472,pixar fun,adventure animation children comedy fantasy
2,3,1.0,4.0,-0.002703,4.040472,moldy old,comedy romance
43,47,1.0,5.0,0.032833,4.040472,mystery twist ending serial killer,mystery thriller
46,50,1.0,5.0,0.081903,4.040472,mindfuck suspense thriller tricky twist ending...,crime mystery thriller
89,101,1.0,5.0,-0.005519,4.040472,crime off-beat comedy quirky,adventure comedy crime romance


In [33]:
y = data_X['rating']
y.shape

(114,)

In [34]:
x1 = (tfidf1.transform(data_X['TAG'])).toarray()
x2 = (tfidf2.transform(data_X['GENRE'])).toarray()
x3 = list(zip(x1, x2, data_X['score_mov'], data_X['mean_mark']))
x3

[(array([0., 0., 0., ..., 0., 0., 0.]),
  array([0.        , 0.3983147 , 0.52247057, 0.51053891, 0.28185923,
         0.        , 0.        , 0.        , 0.47778164, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]),
  0.03492379067107298,
  4.040472175379427),
 (array([0., 0., 0., ..., 0., 0., 0.]),
  array([0.        , 0.        , 0.        , 0.        , 0.64167385,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.76697762, 0.        , 0.        , 0.        , 0.        ]),
  -0.0027027198735167634,
  4.040472175379427),
 (array([0., 0., 0., ..., 0., 0., 0.]),
  array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.79233952, 0.        ,
         0.        , 0. 

In [35]:
XX = []
for i in x3:
    #print(i[0].reshape(-1))
    j = np.insert(np.concatenate((i[0].ravel(), i[1].ravel()), axis=0), [0,1], [i[2], i[3]], axis=None)
    XX.append(j)
    #XX.append(np.concatenate((i[0].ravel(), i[1].ravel()), axis=0))
XX

[array([0.03492379, 0.        , 4.04047218, ..., 0.        , 0.        ,
        0.        ]),
 array([-2.70271987e-03,  0.00000000e+00,  4.04047218e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([0.03283313, 0.        , 4.04047218, ..., 0.61008039, 0.        ,
        0.        ]),
 array([0.08190314, 0.        , 4.04047218, ..., 0.50174155, 0.        ,
        0.        ]),
 array([-0.00551896,  0.        ,  4.04047218, ...,  0.        ,
         0.        ,  0.        ]),
 array([0.161069  , 0.        , 4.04047218, ..., 0.        , 0.77289652,
        0.        ]),
 array([-1.83952100e-04,  0.00000000e+00,  4.04047218e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([0.03288619, 0.        , 4.04047218, ..., 0.        , 0.        ,
        0.        ]),
 array([-0.00530096,  0.        ,  4.04047218, ...,  0.        ,
         0.        ,  0.        ]),
 array([0.48549528, 0.        , 4.04047218, ..., 0.        , 0.        ,


In [36]:
lr = LinearRegression()

In [37]:
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.3, random_state=42)

In [38]:
lr.fit(X_train, y_train)

LinearRegression()

In [39]:
y_pred = lr.predict(X_test)

In [40]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse
#rmse = sqrt(mean_squared_error(y_test, y_pred))

5.075503894832743